In [3]:
import polars as pl
import numpy as np
import pandas as pd
import os

# House 1

## Read all data

In [4]:
house1_data = pl.concat([pl.read_csv("../../data/RAE_dataset/raw_data/house1_subs_blk2.csv"), 
            pl.read_csv("../../data/RAE_dataset/raw_data/house1_subs_blk1.csv")])
len(house1_data)

145637040

In [5]:
transform_columns = ["unix_ts", "Irms", "pf", "P", "Q", "S", "label"]
save_path = "../../data/RAE_dataset/transform_data"

In [6]:
select_columns = ["unix_ts", "sub", "V", "I", "P", "Q", "S"]
house1_data = house1_data.select(select_columns)

In [7]:
house1_data.head()

unix_ts,sub,V,I,P,Q,S
i64,i64,f64,f64,i64,i64,i64
1457251200,1,119.7,0.0,0,0,3
1457251200,2,119.1,0.0,1,0,3
1457251200,3,119.7,0.0,0,0,1
1457251200,4,119.1,0.0,1,2,3
1457251200,5,119.7,0.0,0,0,3


## Transform data

<b>House 1 labels:</b>
-  1 Kitchen Wall Oven L1
-  2 Kitchen Wall Oven L2
-  3 Kitchen Counter Plugs L1
-  4 Kitchen Counter Plugs L2
-  5 Clothes Dryer L1
-  6 Clothes Dryer L2
-  7 Upstairs Bedroom AFCI Arc-Fault Plugs
-  8 Kitchen Fridge
-  9 Clothes Washer
- 10 Kitchen Dishwasher
- 11 Furnace & Hot Water Unit (incl. Furnace Room Plug)
- 12 Basement Plugs & Lights (incl. Outside Plugs)
- 13 Heat Pump L1
- 14 Heat Pump L2
- 15 Garage Sub-Panel L1
- 16 Garage Sub-Panel L2
- 17 Upstairs Plugs & Lights L1 (incl. Bathroom Lights and Vent Fan, Smoke Alarms, Living Room Plugs)
- 18 Upstairs Plugs & Lights L2 (incl. Bathroom Lights and Vent Fan, Smoke Alarms, Living Room Plugs)
- 19 Basement Blue Plugs (incl. Ent. TV/Amp/DVD/PVR)
- 20 Bathrooms (incl. 3 GFCI Plugs, 2 Lights, 1 Vent Fan, Chest Freezer)
- 21 Rental Suite Sub-Panel L1
- 22 Rental Suite Sub-Panel L2
- 23 Misc. Plugs (incl. Dining Room, Gas Cooktop, Microwave)
- 24 Home Office (incl. Telco/Cable/Net/Security Equip.)

In [8]:
meter_ids_dict = {
    "clothes_dryer": [5,6],
    "kitchen_oven": [1,2],
    "kitchen_fridge": [8],
    "heat_pump": [13, 14]
}

In [9]:
for key in meter_ids_dict:
    meter_ids = meter_ids_dict[key]
    print(f"Working on key: {key}")
    if len(meter_ids) == 2:
        load1_df = house1_data.filter(pl.col("sub") == meter_ids[0])
        load2_df = house1_data.filter(pl.col("sub") == meter_ids[1])
        combined_np = load1_df[:, 3:].to_numpy() + load2_df[:, 3:].to_numpy()
        combined_df = pl.DataFrame(combined_np, schema=["Irms", "P", "Q", "S"])
        combined_df = combined_df.with_columns(label=pl.lit(key))
        combined_df = combined_df.with_columns(unix_ts=pl.lit(load1_df["unix_ts"]))
        combined_df = combined_df.with_columns(pf=pl.col("P")/pl.col("S"))
        combined_df = combined_df.with_columns(
            pl.when(combined_df["S"] != 0)
            .then(combined_df["P"]/combined_df["S"])
            .otherwise(0)
            .alias("pf")
        )
        combined_df = combined_df.select(transform_columns)
        print(combined_df.head())
        combined_df.to_pandas().to_csv(os.path.join(save_path, f"house1_{key}.csv"))
    elif len(meter_ids) == 1:
        load_df = house1_data.filter(pl.col("sub") == meter_ids[0])
        load_df = load_df.rename({"I": "Irms"})
        load_df = load_df.with_columns(label=pl.lit(key))  
        load_df = load_df.with_columns(
            pl.when(load_df["S"] != 0)
            .then(load_df["P"]/load_df["S"])
            .otherwise(0)
            .alias("pf")
        )
        load_df = load_df.select(transform_columns)
        print(load_df.head())
        load_df.to_pandas().to_csv(os.path.join(save_path, f"house1_{key}.csv"))

Working on key: clothes_dryer
shape: (5, 7)
┌────────────┬──────┬─────┬─────┬─────┬─────┬───────────────┐
│ unix_ts    ┆ Irms ┆ pf  ┆ P   ┆ Q   ┆ S   ┆ label         │
│ ---        ┆ ---  ┆ --- ┆ --- ┆ --- ┆ --- ┆ ---           │
│ i64        ┆ f64  ┆ f64 ┆ f64 ┆ f64 ┆ f64 ┆ str           │
╞════════════╪══════╪═════╪═════╪═════╪═════╪═══════════════╡
│ 1457251200 ┆ 0.0  ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ 5.0 ┆ clothes_dryer │
│ 1457251201 ┆ 0.0  ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ 5.0 ┆ clothes_dryer │
│ 1457251202 ┆ 0.0  ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ 5.0 ┆ clothes_dryer │
│ 1457251203 ┆ 0.0  ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ 5.0 ┆ clothes_dryer │
│ 1457251204 ┆ 0.0  ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ 5.0 ┆ clothes_dryer │
└────────────┴──────┴─────┴─────┴─────┴─────┴───────────────┘
Working on key: kitchen_oven
shape: (5, 7)
┌────────────┬──────┬──────────┬─────┬─────┬─────┬──────────────┐
│ unix_ts    ┆ Irms ┆ pf       ┆ P   ┆ Q   ┆ S   ┆ label        │
│ ---        ┆ ---  ┆ ---      ┆ --- ┆ --- ┆ --- ┆ ---          │
│ i64        ┆ f64  ┆ f64      ┆ 

# House 2

In [3]:
house2_data = pl.read_csv("../../data/RAE_dataset/raw_data/house2_subs_blk1.csv")
len(house2_data)

104588169

In [4]:
transform_columns = ["unix_ts", "Irms", "pf", "P", "Q", "S", "label"]
select_columns = ["unix_ts", "sub", "V", "I", "P", "Q", "S"]
save_path = "../../data/RAE_dataset/transform_data"

In [5]:
house2_data = house2_data.select(select_columns)
house2_data.head()

unix_ts,sub,V,I,P,Q,S
i64,i64,f64,f64,i64,i64,i64
1505286000,1,120.9,0.9,71,57,111
1505286000,2,121.1,0.3,32,14,40
1505286000,3,121.1,0.0,0,0,0
1505286000,4,120.9,0.0,0,0,0
1505286000,5,121.1,0.0,1,0,3


## Transform data

<b>House 2 label</b>
-  1 House Sub-Panel L1
-  2 House Sub-Panel L2
-  3 Lights & Plugs (general label)
-  4 Clothes Dryer L1
-  5 Clothes Dryer L2
-  6 Bedroom Plugs
-  7 Built-in Vacuum
-  8 Boiler (for hot water and radiant heating)
-  9 Lights & Plugs (general label)
- 10 Clothes Washer
- 11 Kitchen Fridge
- 12 Lights & Plugs (general label, incl. Internet modem and network equipment)
- 13 Bedrooms AFCI Arc-Fault Plugs
- 14 Kitchen Counter Plugs
- 15 Kitchen Counter Plugs
- 16 Lights & Plugs (general label)
- 17 Lights & Plugs (general label)
- 18 Outside Plugs
- 19 Dishwasher
- 20 Lights & Plugs (general label)
- 21 Mobile Phone Changers (garburator & microwave not installed)

In [6]:
house2_meters = {
    "lights_and_plugs_3": [3],
    "lights_and_plugs_16": [16],
    "lights_and_plugs_20": [20]
}

for key in house2_meters:
    meter_ids = house2_meters[key]
    print(f"Working on key: {key}")
    if len(meter_ids) == 2:
        load1_df = house2_data.filter(pl.col("sub") == meter_ids[0])
        load2_df = house2_data.filter(pl.col("sub") == meter_ids[1])
        combined_np = load1_df[:, 3:].to_numpy() + load2_df[:, 3:].to_numpy()
        combined_df = pl.DataFrame(combined_np, schema=["Irms", "P", "Q", "S"])
        combined_df = combined_df.with_columns(label=pl.lit(key))
        combined_df = combined_df.with_columns(unix_ts=pl.lit(load1_df["unix_ts"]))
        combined_df = combined_df.with_columns(
            pl.when(combined_df["S"] != 0)
            .then(combined_df["P"]/combined_df["S"])
            .otherwise(0)
            .alias("pf")
        )
        combined_df = combined_df.select(transform_columns)
        print(combined_df.head())
        combined_df.to_pandas().to_csv(os.path.join(save_path, f"house2_{key}.csv"))
    elif len(meter_ids) == 1:
        load_df = house2_data.filter(pl.col("sub") == meter_ids[0])
        load_df = load_df.rename({"I": "Irms"})
        load_df = load_df.with_columns(label=pl.lit(key))  
        load_df = load_df.with_columns(
            pl.when(load_df["S"] != 0)
            .then(load_df["P"]/load_df["S"])
            .otherwise(0)
            .alias("pf")
        )
        load_df = load_df.select(transform_columns)
        print(load_df.head())
        load_df.to_pandas().to_csv(os.path.join(save_path, f"house2_{key}.csv"))

Working on key: lights_and_plugs_3
shape: (5, 7)
┌────────────┬──────┬─────┬─────┬─────┬─────┬────────────────────┐
│ unix_ts    ┆ Irms ┆ pf  ┆ P   ┆ Q   ┆ S   ┆ label              │
│ ---        ┆ ---  ┆ --- ┆ --- ┆ --- ┆ --- ┆ ---                │
│ i64        ┆ f64  ┆ f64 ┆ i64 ┆ i64 ┆ i64 ┆ str                │
╞════════════╪══════╪═════╪═════╪═════╪═════╪════════════════════╡
│ 1505286000 ┆ 0.0  ┆ 0.0 ┆ 0   ┆ 0   ┆ 0   ┆ lights_and_plugs_3 │
│ 1505286001 ┆ 0.0  ┆ 0.0 ┆ 0   ┆ 0   ┆ 2   ┆ lights_and_plugs_3 │
│ 1505286002 ┆ 0.0  ┆ 0.0 ┆ 0   ┆ 0   ┆ 2   ┆ lights_and_plugs_3 │
│ 1505286003 ┆ 0.0  ┆ 0.0 ┆ 0   ┆ 0   ┆ 2   ┆ lights_and_plugs_3 │
│ 1505286004 ┆ 0.0  ┆ 0.0 ┆ 0   ┆ 0   ┆ 2   ┆ lights_and_plugs_3 │
└────────────┴──────┴─────┴─────┴─────┴─────┴────────────────────┘
Working on key: lights_and_plugs_16
shape: (5, 7)
┌────────────┬──────┬─────┬─────┬─────┬─────┬─────────────────────┐
│ unix_ts    ┆ Irms ┆ pf  ┆ P   ┆ Q   ┆ S   ┆ label               │
│ ---        ┆ ---  ┆ --- ┆ 